# Assignment 9

In [302]:
import numpy as np
import pandas as pd

### Part A

Consider a two period binomial model. Suppose after each period, the stock will either rise in value by ```r``` percent, or fall in value by ```f``` percent. Currently, the stock price is ```P```, and the risk free rate is ```rf```.

Write the function ```european_put```, which calculates the value of a one year European Put option with a strike price of ```K```. 

In [303]:
def european_put(P, K, r, f, rf):
    p = ((1 + rf)**0.5 - 1 - f)/(r - f)
    neg_p = 1 - p
    
    low_price_2 = P*(1 + f)**2
    high_price_2 = P*(1 + r)**2
    
    p1_plus_plus = max(K - high_price_2, 0)
    p1_plus_minus = max(K - P, 0)
    
    p1_minus_plus = max(K - P, 0)
    p1_minus_minus = max(K - low_price_2, 0)
    
    p1_plus = (p * p1_plus_plus + neg_p * p1_plus_minus)/((1 + rf)**0.5)
    p1_minus = (p * p1_minus_plus + neg_p * p1_minus_minus)/((1 + rf)**0.5)
    
    value = (p * p1_plus + neg_p * p1_minus)/((1 + rf)**0.5)
    
    return value
            
# european_put(40,40, 0.25, -0.2, 0.1) should return 2.616...

Write the function ```american_put```, which calculates the value of a one year American Put option with a strike price of ```K```. 

In [305]:
def american_put(P, K, r, f, rf):
    # Write your code here

    value = 0
    
    if (K > P):
        return P - K
    
    p = ((1 + rf)**0.5 - 1 - f)/(r - f)
    inverse_of_p = 1 - p
    
    low = P * (1 + f)
    high = P * (1 + r)
    
    if (low < K): 
        payoff_low = K - low
        high_payoff = max(K - high, 0)
        value = (p * high_payoff + inverse_of_p * payoff_low)/((1 + rf)**0.5)
        return value
    
    value = european_put(P, K, r, f, rt)
    
    return value

# american_put(40,40, 0.25, -0.2, 0.1) should return 3.410...

### Part B

In addition to simple calls and puts, a wide variety of exotic options are traded in over-the-counter markets. In most cases, there are no analytic formulas available to value these options, and so numerical techniques must be used. One example of these techniques are Monte Carlo methods. These are particularly useful in certain contexts, such as when option payoffs depend not only on the stock price at expiration but also upon the history of the stock price sampled at various points during the life of the option. This question will explore some basic examples, and it will require relatively large number of paths. Assume that the stock price will be needed at the end of each of the next 12 months, in addition to S0 today, and start by simulating `N` such stock price paths. In particular:

•	You have been provided the function ```gen_random_vars```, which generates random variables that follow a standard normal distribution (Note that you should never actually call ```gen_random_vars``` in any of your functions, this is simply meant to help give you inputs to help test your code). This outputs a table with `N` rows and 12 columns. Denote the value of the random variable on path i for month j by zi,j where i = 1, . . . ,`N` and j = 1, . . . ,12.

Write the function ```create_paths```, which takes as an argument a DataFrame ```random_vars```, the output of the function ```gen_random_vars```, and ```S0```, the initial stock price.

This function will generate a DataFrame with `N` stock price paths $S_{i,j}$ where $i = 1, . . . ,N$ and $j = 0, . . . ,12$. In each case, $S_{i,0} =$ ```S_0```. For the remaining entries in this table, define $∆t = 1/12$ and set

$$ S_{i,j} = S_{i, j-1} \exp [(r - \sigma^2 / 2) \Delta t + \sigma \sqrt{\Delta t} z_{i,j}]$$

 
for i = 1, . . . ,`N` and j = 1, . . . ,12. (If you are wondering why we are using the risk-free rate r here instead of the expected rate of return on the underlying stock, it is because we are using risk-neutral valuation.) 

Note that throughout the rest of this question, $S_T$ on each simulated path i is defined to be equal to $S_{i,12}$.



In [307]:
def gen_random_vars(SEED, N):
    rng = np.random.default_rng(SEED)

    zArr = np.array([rng.normal(size=N)])
    for i in range(11):
        z = rng.normal(size=N)
        zArr = np.r_[zArr,[z]]
    

    randomVariables = pd.DataFrame(zArr.transpose())
    return randomVariables

r_vars = gen_random_vars(727, 10000)
r_vars

0         1         2         3         4         5         6   \
0     1.744763 -0.514250 -0.710679  1.495061  0.413633 -2.361391  0.122873   
1    -0.962309 -0.576888  0.396051  1.595821  0.008168 -0.348428 -0.340074   
2     0.496945  0.579497 -0.315875  0.330535 -0.045317  0.722330  0.760781   
3    -0.282490 -0.940921  1.175841  0.405341 -0.923402  0.977314 -0.128823   
4    -0.400287 -1.035328 -1.143469 -0.976739  0.649610 -0.414282 -0.257923   
...        ...       ...       ...       ...       ...       ...       ...   
9995  0.812176  0.492083  0.234801 -0.822796 -1.171159 -1.472986  0.598065   
9996 -0.714678 -0.710493 -0.644405  0.711664 -0.901012  0.526393 -2.218868   
9997 -1.684857  0.816217  0.048860  0.366472  0.296956 -0.250303  1.962326   
9998 -0.418933  0.839082 -2.012594  1.065439 -0.527060 -0.770415 -0.886027   
9999 -0.471454  0.268310  0.767433 -1.018053 -1.031914 -0.759433  0.806048   

            7         8         9         10        11  
0    -0.095099 -0.884679 -0.148495 -0.508705  1.562580  
1     0.230534 -0.556868 -1.567693 -0.009760  0.885091  
2    -0.070977  0.250281 -0.329167  0.407775  0.864023  
3    -0.688989 -1.411402  0.010026 -1.573026  0.730829  
4     0.762621 -1.664448 -1.159700  0.268761  1.822126  
...        ...       ...       ...       ...       ...  
9995 -0.239307 -0.822385  0.653382  0.729384  0.928624  
9996  2.422271  0.392027  1.610234  0.387898 -0.058958  
9997  1.416475  0.541789  0.660781  1.132385  0.190961  
9998  0.911825 -0.648514 -2.227339 -0.125608 -1.135578  
9999  0.445378  0.758105  0.378315 -0.898852  0.473172  

[10000 rows x 12 columns]

In [308]:
def create_paths(random_vars, S0, rf, sigma):
    # Do not rename the columns or rows of the outputted dataframe. 
    # The outputted DataFrame should have N rows and 13 columns.
    T = 1/12
    
    first_column = [S0 for _ in range(len(random_vars))]
    
    df = pd.DataFrame(data = {"0": first_column})
    
    df = pd.concat([df, random_vars], axis=1)
    
    for row in range(len(random_vars)):
        for col in range(1, len(df.iloc[row])):
            z = df.iloc[row].iloc[col]
            df.loc[row, df.columns[col]] = df.loc[row, df.columns[col - 1]] * np.exp((rf - (sigma**2/2)) * T + sigma * T**0.5 * z) 
            
    column_names = list(range(13)) 
    df.columns = column_names
    
    return df

# If we let sample_paths = create_paths(r_vars, 72.25, 0.04 , 0.35),
# The last column of the first row of sample_paths should equal 71.56...

•	Write the function ```payoff_table```, which returns another DataFrame with `N` rows and 6 columns. The columns will be used to store discounted payoffs at expiration for various different options, as indicated below:

–	European call option, with payoff $max(S_T − K,0)$.

–	European put option, with payoff $max(K - S_T ,0)$.

–	Average price put option, with payoff $max(K − S^-,0)$, where
$S¯ = S_0 + S_1 + · · · + S_{12}$ .

–	Floating lookback call option, with payoff $S_T - S_{min}$, where $S_{min}$ on each path is the minimum of the stock prices $S_0, S_1, . . . , S_{12}$ on that path.

–	Up-and-out call option with barrier ```B```, with payoff $max(S_T - K,0)$ as long as none of the observed monthly stock prices on the given path are greater than or equal to ```B```. If any of the monthly stock prices are greater than or equal to B, then the payoff is zero.

–	Up-and-in call option with barrier ```B``` , with payoff $max(S_T - K,0)$ as long as at least one of the observed monthly stock prices on the given path is greater than or equal to ```B```. If this condition is not satisfied, the option payoff is zero.

Note that “discounted payoffs” implies that the entries in this table should have the payoff functions multiplied by $e^{−rT}$ , where T = 1.

This function will also take in a dataframe ```paths```, which is the output from the previous function ```create_paths```.

In [310]:
def payoff_table(paths, K, T, rf, B):
    
    euro_call = []
    euro_put = []
    avg_price_cut = []
    float_lookbook = []
    up_and_out = []
    up_and_in = []
    
    for row in paths.index:
        euro_call.append(max(paths.iloc[row].iloc[12]- K, 0) * np.exp(-rf*T))
        euro_put.append(max(K - paths.iloc[row].iloc[12], 0) * np.exp(-rf*T))
        avg_price_cut.append(max(K - paths.iloc[row].mean(), 0) * np.exp(-rf*T))
        float_lookbook.append((paths.iloc[row].iloc[12] - paths.iloc[row].min()) * np.exp(-rf*T))
        
        bool1 = False
        bool2 = False
        for col in range(len(paths.iloc[row])):
            if paths.loc[row, paths.columns[col]] >= B:
                
                if (not bool1):
                    up_and_out.append(0)
                    bool1 = True
                
                if (not bool2):
                    up_and_in.append(max(paths.iloc[row].iloc[12]- K, 0) * np.exp(-rf*T))
                    bool2 = True
        
        if not bool1:
            up_and_out.append(max(paths.iloc[row].iloc[12]- K, 0) * np.exp(-rf*T))
            
        if not bool2:
            up_and_in.append(0)
        
    answer = pd.DataFrame(data = {
        "European Call" : euro_call,
        "European Put" : euro_put,
        "Average Price Put" : avg_price_cut,
        "Floating Lookback" : float_lookbook,
        "Up-and-out" : up_and_out,
        "Up-and-in" : up_and_in
    })
    
    return answer
            
# If we let sample_payoffs = payoff_table(sample_paths, 70, 1, 0.04, 90)
# The payoff of the first European Call should be 1.500...

In [311]:
sample_payoffs = payoff_table(sample_paths, 70, 1, 0.04, 90)
sample_payoffs

European Call  European Put  Average Price Put  Floating Lookback  \
0          1.500689      0.000000           0.000000           9.937430   
1          0.000000      7.338567           2.899000           5.028425   
2         31.017676      0.000000           0.000000          28.855900   
3          0.000000     15.259282           3.797994           3.615473   
4          0.000000     19.775742          16.035582           8.905278   
...             ...           ...                ...                ...   
9995       0.154346      0.000000           1.203830          13.755086   
9996       6.438700      0.000000           5.151621          27.700383   
9997      51.182324      0.000000           0.000000          59.990036   
9998       0.000000     29.949076          10.683546           0.000000   
9999       0.000000      1.213008           2.458833          11.298147   

      Up-and-out  Up-and-in  
0       0.000000   1.500689  
1       0.000000   0.000000  
2       0.000000  31.017676  
3       0.000000   0.000000  
4       0.000000   0.000000  
...          ...        ...  
9995    0.154346   0.000000  
9996    6.438700   0.000000  
9997    0.000000  51.182324  
9998    0.000000   0.000000  
9999    0.000000   0.000000  

[10000 rows x 6 columns]

Write the function ```statistics```, which takes as input a DataFrame ```payoffs```, formatted as one produced by ```payoff_table```, and displays the following statistics for each of the options above (below N is the number of simulated paths):

•	Mean discounted payoff µ. This is the estimated option premium.

•	Standard error of option premium ϕ. This is the standard deviation of the option payoff divided by √N .

•	Lower bound for 95% confidence interval for option premium, which is µ - 1.96ϕ.

•	Upper bound for 95% confidence interval for option premium, which is µ + 1.96ϕ.

The size of the returned DataFrame should have 6 rows and 4 columns. 

In [312]:
def statistics(payoffs):
    mean = []
    std_err = []
    low_bound = []
    upp_bound = []
    
    N = len(payoffs)
    
    for col in payoffs.columns:
        mean.append(payoffs[col].mean())
        std_err.append(((payoffs[col].std())/N**0.5))
        low_bound.append(payoffs[col].mean() - 1.96* ((payoffs[col].std())/N**0.5)) # mean payoff - 1.96 * std err
        upp_bound.append(payoffs[col].mean() + 1.96 * ((payoffs[col].std())/N**0.5))
    
    # Note that the index rows have been renamed here. Ensure that your rows match the order 
    # that the indices have been renamed as.
    output = pd.DataFrame(data = {
        "Mean" : mean,
        "Standard Error" : std_err,
        "Lower Bound" : low_bound,
        "Upper Bound" : upp_bound
    }).rename(index={0: "European Call", 1: "European Put", 2: "Average Price Put", 3: "Floating Lookback",4: "Up-and-out",5: "Up-and-in"}, errors="raise")
    
    return output

# If we call statistics(sample_payoffs), the mean of the European Puts should be 7.370...

In [313]:
statistics(sample_payoffs)

Mean  Standard Error  Lower Bound  Upper Bound
European Call      12.093962        0.190808    11.719979    12.467945
European Put        7.370213        0.102129     7.170040     7.570387
Average Price Put   3.833641        0.058521     3.718940     3.948342
Floating Lookback  15.992686        0.184633    15.630806    16.354566
Up-and-out          1.116772        0.032983     1.052125     1.181419
Up-and-in          10.977190        0.194349    10.596265    11.358115

One disadvantage of Monte Carlo methods is that they can be slow to converge. On the other hand, there are situations where they are the only methods that can be used. As a result, there has been a lot of research into variance reduction methods, in an effort to improve the rate of convergence. 

One of the simplest such approaches is called antithetic variates. This relies on the fact that if Z is a standard normal random variable, then so is -Z. To see how this can be used, generate another `N` stock price paths, but this time using $-z_{i,j}$ instead of $z_{i,j}$ throughout.

In order to accomplish this, write the function ```create_negative_paths```. This function has the same arguments as ```create_paths```, but creates the stock prices in the manner described above.

In [314]:
def create_paths(random_vars, S0, rf, sigma):
    # Do not rename the columns or rows of the outputted dataframe. 
    # The outputted DataFrame should have N rows and 13 columns.
    T = 1/12
    
    first_column = [S0 for _ in range(len(random_vars))]
    
    df = pd.DataFrame(data = {"0": first_column})
    
    df = pd.concat([df, random_vars], axis=1)
    
    for row in range(len(random_vars)):
        for col in range(1, len(df.iloc[row])):
            z = df.iloc[row].iloc[col]
            df.loc[row, df.columns[col]] = df.loc[row, df.columns[col - 1]] * np.exp((rf - (sigma**2/2)) * T + sigma * T**0.5 * -z) 
            
    column_names = list(range(13)) 
    df.columns = column_names
    
    return df

In [315]:
sample_neg_paths = create_negative_paths(r_vars, 72.25, 0.04 , 0.35)
sample_neg_paths

0          1          2          3           4          5   \
0     72.25  60.465753  63.577791  68.189988   58.526026  56.031205   
1     72.25  79.486690  84.108312  80.666154   68.532797  68.355108   
2     72.25  68.590360  64.575235  66.551425   64.251695  64.432357   
3     72.25  74.210318  81.467120  72.213429   69.193092  75.824942   
4     72.25  75.098829  83.232666  93.260903  102.751789  96.054071   
...     ...        ...        ...        ...         ...        ...   
9995  72.25  66.440199  63.105833  61.517331   66.731719  74.981325   
9996  72.25  77.522632  83.144884  88.581420   82.289821  89.973169   
9997  72.25  85.506572  78.598673  78.073237   75.102175  72.753380   
9998  72.25  75.240445  69.002336  84.412421   75.663486  79.660752   
9999  72.25  75.640770  73.487516  67.884389   75.105632  83.211491   

             6           7          8           9           10          11  \
0     71.003142   70.003039  70.553850   77.014221   78.040090   82.010686   
1     70.679061   73.020365  71.212991   75.201283   87.952012   87.883026   
2     59.791504   55.269779  55.569065   54.085619   55.815707   53.468059   
3     68.573922   69.349387  74.217457   85.441449   85.203922   99.704355   
4     99.982779  102.440970  94.676278  111.816859  125.494670  121.916682   
...         ...         ...        ...         ...         ...         ...   
9995  86.859623   81.621784  83.471396   90.542914   84.608761   78.458725   
9996  85.162046  106.375067  83.134775   79.764879   67.668469   64.952595   
9997  74.484737   60.980692  52.755658   49.857047   46.554618   41.448154   
9998  85.956909   93.840463  85.430044   91.053788  113.831767  115.081636   
9999  89.688714   82.527704  78.756737   72.820452   69.965513   76.481459   

              12  
0      69.909489  
1      80.222939  
2      48.911668  
3      92.443557  
4     101.237194  
...          ...  
9995   71.305760  
9996   65.225063  
9997   40.584188  
9998  128.844406  
9999   72.782091  

[10000 rows x 13 columns]

Write the function ```antithetic_statistics```.

In this function, you will use ```payoff_table``` to create two tables, one using paths generated from ```create_paths```, and one generated from ```create_negative_paths```.

Next, create a third table which contains the average payoff between the two tables.

Finally, call ```statistics``` on this third table. Return the table produced by ```statistics```.

If you compare this table with the output of `statistics` (as above, without creating two tables), you should notice that Mean, Lower Bound, and Upper Bound columns should have relatively similar values, while the Standard Error column has decreased significantly.

In [320]:
def antithetic_statistics(random_vars, S0, rf, sigma, K, T, B):
    # Write your code here
    
    # create the dataframe of prices
    create_path_price = create_paths(random_vars, S0, rf, sigma)
    create_negative_paths_price = create_negative_paths(random_vars, S0, rf, sigma)
    
    path_payoff = payoff_table(create_path_price, K, T, rf, B)
    path_payoff_neg = payoff_table(create_negative_paths_price, K, T, rf, B)
    
    avg_payoff = path_payoff.copy()
    
    for row in path_payoff.index:
        for col in range(len(path_payoff.iloc[row])):
            avg_payoff.loc[row, avg_payoff.columns[col]] = np.mean([path_payoff.loc[row, path_payoff.columns[col]], path_payoff_neg.loc[row, path_payoff_neg.columns[col]]])
    
    
    avg_stat_table = statistics(avg_payoff)
    
    return avg_stat_table

